**Here is an example of using shingling to efficiently estimate whether two documents are near duplicates. **

See [Chapter 19](http://nlp.stanford.edu/IR-book/pdf/19web.pdf) of your book for more details.


In [190]:
doc1 = 'a rose is a rose is a rose'.split()    # Gertrude Stein
doc2 = 'a rose is a rose is an onion'.split()  # Ernest Hemmingway

In [191]:
# Represent each document as a *set* of ngrams.

def shingle(doc, n):
    return set(' '.join(doc[i:i+n]) for i in range(len(doc) - n + 1))

print(shingle(doc1, 4))
print(shingle(doc2, 4))

{'rose is a rose', 'a rose is a', 'is a rose is'}
{'rose is a rose', 'a rose is an', 'rose is an onion', 'a rose is a', 'is a rose is'}


** Similarity of two documents $\propto$ number of overlapping shingles. **

In [193]:
# Jaccard similarity: size of intersection over size of union.
def jaccard(set1, set2):
    return len(set1 & set2) / len(set1 | set2)

jaccard(set([1,2,3]), set([2,3,4]))
# |{2,3}| / |{1,2,3,4}|

0.5

In [194]:
jaccard(shingle(doc1, 4), shingle(doc2, 4))

0.6

In [195]:
shingle(doc1, 4) & shingle(doc2, 4)

{'a rose is a', 'is a rose is', 'rose is a rose'}

In [196]:
shingle(doc1, 4) | shingle(doc2, 4)

{'a rose is a',
 'a rose is an',
 'is a rose is',
 'rose is a rose',
 'rose is an onion'}

**How can we compute Jaccard for all pairs of documents?**

$O(n^2)$, where $n \approx 1B$ web pages

To compute Jaccard for each pair of documents, we need to compute intersection of all their shingles: $O(k)$, where $k$ is number of tokens in a document.

So, total time is  
$O((Kn)^2)$, where $K$ is typical document length.

Can we get by without comparing all shingles?  
i.e., can we reduce the size of $K$?

**MinHash Trick**

Idea: used a small, fixed set of values to summarize a document.

In [197]:
# Hash each string to a 64-bit number.
hash('rose')
# Small chance of collision.

8686785072249457076

In [198]:
# Now, for each doc, we convert its set of shingles to a set of hash values, one per ngram.
def hash_doc(doc):
    return set(hash(s) for s in shingle(doc, 4))
hash_doc(doc1)

{-8757600265101086734, 1747002848098801166, 1935487005774968538}

In [199]:
jaccard(hash_doc(doc1), hash_doc(doc2))

0.6

**We've saved space (string->int), but not much time (still same number of comparisons)**

**Idea:** Use a small, fixed number of random hashes to represent each document.

In [200]:
# Generate a random string of fixed size.
import random
import string

def random_string(size, seed):
    random.seed(seed)
    return ''.join(random.choice(string.ascii_lowercase)
                   for _ in range(size))

random_string(5, 123)

'bicyn'

In [201]:
random_string(5, 123)

'bicyn'

In [202]:
random_string(5, 42)

'udaxi'

In [176]:
# Now, prepend a random string to each ngram prior to hashing.
# This results in M different sets of hash values.
def hash_doc_random(doc, seeds):
    sets = []
    for seed in seeds:
        prefix = random_string(5, seed)
        sets.append(set([hash(prefix + s) for s in shingle(doc, 4)]))
    return sets

# E.g., for M=2
hash_doc_random(doc1, [123, 456])
# Here, row i is the output of the ith hashing function.

[{-7455963190061466951, -5961080585398496229, 2527587970878870217},
 {-3587527669377022179, 750713077338955787, 1588185390600274052}]

** Now, summarize these M different sets by taking the minimum value from each (somewhat arbitrary). **

In [203]:
# M=20: Create 20 random seeds for 20 different hash functions.
# (In practice, we would use a larger value of M.)
random.seed(42)
seeds = [random.randint(0, 1e10) for _ in range(20)]
seeds

[2746317213,
 8697354961,
 1181241943,
 958682846,
 3163119785,
 8963334018,
 1812140441,
 127978094,
 939042955,
 8703905715,
 9443935785,
 6635473142,
 5241752544,
 6825844140,
 3460967357,
 7293453178,
 5756332150,
 667779376,
 1445662585,
 4693307665]

In [204]:
# Prepend a random string to each ngram prior to hashing.
# This results in K different sets of hash values.
def min_hash(doc, seeds):
    result = set()
    for seed in seeds:
        prefix = random_string(5, seed)
        result.add(min([hash(prefix + s) for s in shingle(doc, 4)]))
    return result

min_hash(doc1, seeds)
# Output is of length M (=20)

{-8339764253898357490,
 -8331239421977272583,
 -7941353375205134921,
 -7471063588538754369,
 -7329917667602004428,
 -7122975656568478326,
 -7105912375894461766,
 -6822862262083637827,
 -6612366677129883046,
 -5643788513242149421,
 -5423508810653492103,
 -5020890668431292256,
 -2680650638414010812,
 -2519516617808959957,
 -434632871186134070,
 426531534575266359,
 1667459092347916612,
 2263830750172537746,
 2982601731111424032,
 5484842312009809327}

In [205]:
# Now, we compute Jaccard of this reduced representation.
jaccard(min_hash(doc1, seeds), min_hash(doc2, seeds))

0.3333333333333333

**Of course, in this example, we've *increased* the values to compare, because the documents are so short. In practice, want M << K**

In [206]:
# Should be able to distinguish between, e.g.:
doc3 = "a rose is a flower is a plant ".split()
jaccard(min_hash(doc1, seeds), min_hash(doc3, seeds))

0.08108108108108109

In [207]:
jaccard(min_hash(doc2, seeds), min_hash(doc3, seeds))

0.08108108108108109

Quality of Jaccard estimate will improve with length of documents (k) and number of random hash functions (M)

** So, to compare two documents, rather than $O(2k)$ time to compare shingles, requires only $O(2M)$ time to compare min_hashed shingles.**

**...but, we still need to do this in the worst case for all pairs of documents, $O(n^2)$**

** Ways to reduce $O(n^2)$ comparisons:**

- Take a hash of the hashes:  
  $H_1 = hash(h_1, h_2, h_3, h_4)$, $H_2 = hash(h_5, h_6, h_7, h_8) \ldots$
- Only consider documents that match in at least one "hash of hashes"